[Home](../../README.md)

### Data Wrangling

This is a demonstration of data wrangling using [Pandas](https://pandas.pydata.org/) the library for data analysis and manipulation.

This Jupyter Notepad demonstrates different processes you can apply to your data to prepare it for feature engineering and model training. For this demonstration we will wrangle the diabetes data set you previewed in the last Jupyter Notebook.

> [!Note]
> None of these processes are destructive to the source CSV as long as you save the modified data to a new CSV.

#### Load the required dependencies

In [17]:
# Import frameworks
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [18]:
data_frame = pd.read_csv("2.1.2.Diabeties_Sample_Data.csv")

#### Dealing with null values

Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The `isnull().sum()` method call returns the null values in any column.

In [19]:
data_frame.isnull().sum()

DoB       0
DoT       0
SEX       1
BMI       0
BP        0
TC        0
BGU       0
FDR       0
Target    1
dtype: int64

If you have null data there are many ways to deal with the empty/null values. These are the two most common approaches.
1. Remove any row with a null value with a `dropna()` method call.
2. Replace missing values with another value with a `fillna()` method call. Generally, we use mean value for numerical columns because it may cause minimal changes in your mathematical analysis while maintaining the original size of the data.

Students should reflect why this example removes the null 'SEX' but replacing the mean 'Target'?

In [20]:
# Remove Null values
data_frame = data_frame.dropna(subset=['SEX'])
data_frame.isnull().sum()

DoB       0
DoT       0
SEX       0
BMI       0
BP        0
TC        0
BGU       0
FDR       0
Target    1
dtype: int64

In [21]:
# Replace Null values with the mean value for the column
data_frame['Target'] = data_frame['Target'].fillna(data_frame['Target'].mean())
data_frame.isnull().sum()

DoB       0
DoT       0
SEX       0
BMI       0
BP        0
TC        0
BGU       0
FDR       0
Target    0
dtype: int64

#### Remove Duplicates

Duplicate data can have detrimental effects on your machine learning models and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or biased models.

The `duplicated().sum()` method call returns the count of duplicate rows in the data frame.

In [22]:
data_frame.duplicated().sum()

np.int64(5)

The `drop_duplicates()` method call can be then stored back onto the data_frame variable removing the duplicates.

In [23]:
data_frame = data_frame.drop_duplicates()
data_frame.duplicated().sum()

np.int64(0)

#### Replace data

We can run a lambda function on a column to modify its values. For a simple example, let’s convert the Sex to lowercase. To run a function over a complete column, we can use the apply method which iterates over each row and modifies the values.

In [24]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda x: x.lower())
data_frame['SEX'].head()

0    female
1    female
2      male
3      male
4      male
Name: SEX, dtype: object

We can check that there are no data entry errors by the `unique()` method call.

In [25]:
data_frame['SEX'].unique()

array(['female', 'male', 'girl'], dtype=object)

In [26]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: 'male' if gender.lower() == 'male' else 'female')
data_frame['SEX'].unique()

array(['female', 'male'], dtype=object)

#### Remove outliers

Outliers can skew your analysis on numerical columns, and it is important to remove them. We can use the 25th and 75th quartile on numerical data, to get the inter-quartile range. This allows us to estimate an acceptable range, and we can then filter out any values outside this range. Mathematically, outliers are values occurring outside 1.5 times the interquartile range (IQR) from the first quartile (Q1) or third quartile (Q3).

In [37]:
#get the inter-quartile range on the salary column
print(data_frame['Target'].describe())
Q1 = data_frame['Target'].quantile(0.25)
Q3 = data_frame['Target'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a BMI above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')


count    425.000000
mean     149.807797
std       75.798750
min       25.000000
25%       86.000000
50%      139.000000
75%      202.000000
max      341.000000
Name: Target, dtype: float64
Outliers are a BMI above 376.0 or below -88.0


In [38]:
# Filter salaries within the acceptable range
data_frame = data_frame[(data_frame['Target'] >= Q1 - 1.5 * IQR) & (data_frame['Target'] <= Q3 + 1.5 * IQR)]
print(data_frame['Target'].describe())

count    425.000000
mean     149.807797
std       75.798750
min       25.000000
25%       86.000000
50%      139.000000
75%      202.000000
max      341.000000
Name: Target, dtype: float64


#### Scaling features to a common range

Scaling the features makes it easier for machine learning algorithms to find the optimal solution, as the different scales of the features do not influence them.

In [39]:
print(data_frame['BMI'].describe())

count    425.000000
mean      26.206824
std        4.243155
min       18.000000
25%       23.100000
50%       25.600000
75%       29.000000
max       38.300000
Name: BMI, dtype: float64


In [ ]:
scale_feature = 'BMI'

#the minimum value with space for outliers
MIN_BP = 20

#the maximum value with space for outliers
MAX_BP = 345

#scale features
data_frame[scale_feature] = [(X - MIN_BP) / (MAX_BP - MIN_BP) for X in data_frame[scale_feature]]

data_frame.describe()

,BMI,BP,TC,BGU,FDR,Target
count,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000
mean,0.431868,0.462088,0.338025,0.466628,0.265808,0.399599
std,0.163305,0.176844,0.141887,0.177612,0.207437,0.233291
min,0.115385,0.038961,0.111111,0.050000,0.000000,0.015385
25%,0.313462,0.331169,0.222222,0.341667,0.000000,0.201538
50%,0.411538,0.441558,0.333333,0.466667,0.250000,0.366154
75%,0.542308,0.597403,0.444444,0.583333,0.500000,0.566154
max,0.896154,0.961039,0.898889,0.950000,0.750000,0.987692


> [!important]
> You need to save the calculations for each dataset you scale for scaling new values for prediction.

#### Save the wrangled data to CSV

In [31]:
data_frame.to_csv('../2.2.Feature_Engineering/2.2.1.wrangled_data.csv', index=False)